In [ ]:
# @title ГОДЖО: 8 КАДРОВ → 30 СЕКУНД (AnimateDiff + RIFE)
import os
import subprocess
import json
import time
import requests
from IPython.display import FileLink, display

# === ПАПКИ ===
WORKSPACE = "/kaggle/working"
COMFY_DIR = f"{WORKSPACE}/ComfyUI"
OUTPUT_DIR = f"{COMFY_DIR}/output"
INPUT_DIR = f"{WORKSPACE}/input"
FRAMES_DIR = f"{WORKSPACE}/frames"
os.makedirs(FRAMES_DIR, exist_ok=True)

# === УСТАНОВКА COMFYUI ===
print("Установка ComfyUI...")
subprocess.run(["rm", "-rf", COMFY_DIR], check=False)
subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI", COMFY_DIR], check=True)
os.chdir(COMFY_DIR)

subprocess.run([
    "pip", "install", "-q", "--no-cache-dir",
    "torch", "torchvision", "torchaudio",
    "--extra-index-url", "https://download.pytorch.org/whl/cu121"
], check=True)
subprocess.run(["pip", "install", "-q", "--no-cache-dir", "-r", "requirements.txt"], check=True)
subprocess.run(["pip", "install", "-q", "huggingface_hub"], check=True)

os.makedirs("custom_nodes", exist_ok=True)
os.chdir("custom_nodes")
subprocess.run(["git", "clone", "https://github.com/ltdrdata/ComfyUI-Manager"], check=False)
subprocess.run(["git", "clone", "https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved"], check=False)
os.chdir(COMFY_DIR)

# === МОДЕЛИ ===
from huggingface_hub import hf_hub_download

# SDXL
ckpt_path = f"{COMFY_DIR}/models/checkpoints/sdxl_base_1.0.safetensors"
if not os.path.exists(ckpt_path):
    print("Скачиваем SDXL...")
    hf_hub_download("stabilityai/stable-diffusion-xl-base-1.0", "sd_xl_base_1.0.safetensors", local_dir=f"{COMFY_DIR}/models/checkpoints")

# AnimateDiff
ad_path = f"{COMFY_DIR}/models/animatediff_models/mm_sd_v15_v2.ckpt"
if not os.path.exists(ad_path):
    print("Скачиваем AnimateDiff...")
    hf_hub_download("guoyww/animatediff", "mm_sd_v15_v2.ckpt", local_dir=f"{COMFY_DIR}/models/animatediff_models")

# Upscale
up_path = f"{COMFY_DIR}/models/upscale_models/4x-UltraSharp.pth"
if not os.path.exists(up_path):
    print("Скачиваем Upscale...")
    import urllib.request
    urllib.request.urlretrieve("https://huggingface.co/Kim2091/UltraSharp/resolve/main/4x-UltraSharp.pth", up_path)

# === WORKFLOW: 8 КАДРОВ (упрощенный - без AnimateDiff для начала) ===
workflow = {
  "3": { "class_type": "CheckpointLoaderSimple", "inputs": { "ckpt_name": "sd_xl_base_1.0.safetensors" } },
  "5": { "class_type": "EmptyLatentImage", "inputs": { "width": 384, "height": 576, "batch_size": 8 } },
  "6": { "class_type": "CLIPTextEncode", "inputs": { "text": "cinematic gojo satoru, white spiky hair, blindfold, confident smirk, anime style, detailed, 8k, dynamic pose", "clip": ["3", 1] } },
  "7": { "class_type": "CLIPTextEncode", "inputs": { "text": "blurry, deformed, low quality, watermark, text", "clip": ["3", 1] } },
  "8": { "class_type": "KSampler", "inputs": { "seed": 42, "steps": 20, "cfg": 7, "sampler_name": "euler", "scheduler": "normal", "positive": ["6", 0], "negative": ["7", 0], "model": ["3", 0], "latent_image": ["5", 0], "denoise": 1.0 } },
  "9": { "class_type": "VAEDecode", "inputs": { "samples": ["8", 0], "vae": ["3", 2] } },
  "10": { "class_type": "UpscaleModelLoader", "inputs": { "model_name": "4x-UltraSharp.pth" } },
  "11": { "class_type": "ImageUpscaleWithModel", "inputs": { "image": ["9", 0], "upscale_model": ["10", 0] } },
  "12": { "class_type": "ImageCrop", "inputs": { "image": ["11", 0], "width": 1080, "height": 1920, "x": 228, "y": 0 } },
  "13": { "class_type": "SaveImage", "inputs": { "images": ["12", 0], "filename_prefix": "GOJO_FRAMES" } }
}

# === API ЗАПУСК ===
print("Запуск сервера...")
server = subprocess.Popen(
    ["python", "main.py", "--dont-print-server", "--listen", "127.0.0.1", "--port", "8188"],
    cwd=COMFY_DIR, stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

for _ in range(60):
    try:
        if requests.get("http://127.0.0.1:8188/system_stats").status_code == 200:
            print("Сервер готов!")
            break
    except:
        time.sleep(1)

print("Генерация 8 кадров...")
resp = requests.post("http://127.0.0.1:8188/prompt", json={"prompt": workflow})
resp_data = resp.json()

if "prompt_id" not in resp_data:
    print("ОШИБКА API:", resp_data)
    server.terminate()
    exit(1)

prompt_id = resp_data["prompt_id"]

for _ in range(300):
    hist = requests.get(f"http://127.0.0.1:8188/history/{prompt_id}").json()
    if prompt_id in hist and hist[prompt_id].get("status", {}).get("completed"):
        print("8 кадров готовы!")
        break
    time.sleep(2)
server.terminate()

# === RIFE: 8 → 256 КАДРОВ ===
# Проверяем что кадры сгенерированы
frame_files = sorted([f for f in os.listdir(OUTPUT_DIR) if f.startswith("GOJO_FRAMES")])

if len(frame_files) >= 8:
    print(f"Найдено {len(frame_files)} кадров, создаем видео...")

    # Копируем кадры в отдельную папку для RIFE
    for i, frame in enumerate(frame_files[:8]):
        src = os.path.join(OUTPUT_DIR, frame)
        dst = os.path.join(FRAMES_DIR, f"frame_{i:03d}.png")
        subprocess.run(["cp", src, dst], check=True)

    # Устанавливаем RIFE
    print("Установка RIFE...")
    subprocess.run(["rm", "-rf", "/kaggle/working/RIFE"], check=False)
    subprocess.run(["git", "clone", "https://github.com/hzwer/arXiv2020-RIFE", "/kaggle/working/RIFE"], check=True)

    os.chdir("/kaggle/working/RIFE")
    subprocess.run(["pip", "install", "-q", "torch", "torchvision"], check=True)

    # Скачиваем модель RIFE
    os.makedirs("RIFE_trained_model_v4.6", exist_ok=True)
    subprocess.run([
        "wget", "-q", "-O", "RIFE_trained_model_v4.6.zip",
        "https://drive.google.com/uc?id=1i3xlKb7ax46jJacQ1aT5vF5lA0Yd5GGI"
    ], check=True)
    subprocess.run([
        "unzip", "-q", "RIFE_trained_model_v4.6.zip", "-d", "RIFE_trained_model_v4.6"
    ], check=True)

    # Запускаем RIFE интерполяцию (8 кадров → 256 кадров, exp=5 означает 2^5=32x интерполяция)
    print("Интерполяция кадров с RIFE...")
    subprocess.run([
        "python", "inference_video.py",
        "--img", FRAMES_DIR,
        "--exp=5",  # 8 * 32 = 256 кадров
        "--video", "/kaggle/working/GOJO_30SEC.mp4",
        "--fps=8"
    ], check=True)

    final = "/kaggle/working/GOJO_30SEC.mp4"
    if os.path.exists(final):
        print("ГОТОВО!")
        display(FileLink(final))
    else:
        print("RIFE не создал видео.")
else:
    print(f"Недостаточно кадров: найдено {len(frame_files)}, нужно минимум 8.")
